In [83]:
import numpy as np

class HopfieldNetwork:
    def __init__(self, num_neurons, threshold_mode='zero', threshold_value=0):
        """
        Initialize the Hopfield Network.

        Parameters:
        -----------
        num_neurons : int
            Number of neurons in the Hopfield network (K).
        threshold_mode : str
            One of {'zero', 'constant', 'initial'}, determines how thresholds are set.
            'zero'     : All thresholds are zero.
            'constant' : All thresholds are set to threshold_value.
            'initial'  : Thresholds are set equal to the initial state x_k(t=0).
        threshold_value : int
            The constant threshold value used if threshold_mode='constant'.
        """
        self.K = num_neurons
        self.W = np.zeros((self.K, self.K), dtype=int)  # Integer weights
        self.thresholds = np.zeros(self.K, dtype=int)
        self.threshold_mode = threshold_mode
        self.threshold_value = threshold_value

    def _check_pattern_shape(self, pattern):
        if pattern.size != self.K:
            raise ValueError("Pattern size does not match number of neurons")
        if not np.all(np.isin(pattern, [-1, 1])):
            raise ValueError("Pattern elements must be either +1 or -1")

    def train_autoassociator(self, patterns):
        """
        Train the network using Hebbian learning rule (autoassociator).
        patterns: list or array of shape (P, K)
            Each pattern is a 1D array with values in {-1, +1}.
        """
        patterns = np.array(patterns)
        for p in patterns:
            self._check_pattern_shape(p)

        # Hebbian learning: W = sum over patterns (x^p * (x^p)^T), with zero diagonal.
        self.W = np.zeros((self.K, self.K), dtype=int)
        for p in patterns:
            self.W += np.outer(p, p)
        # Make diagonal zero
        np.fill_diagonal(self.W, 0)

    def set_thresholds(self, initial_state=None):
        """
        Set thresholds according to the chosen threshold_mode.
        If threshold_mode='initial', initial_state must be provided.
        """
        if self.threshold_mode == 'zero':
            self.thresholds = np.zeros(self.K, dtype=int)
        elif self.threshold_mode == 'constant':
            self.thresholds = np.full(self.K, self.threshold_value, dtype=int)
        elif self.threshold_mode == 'initial':
            if initial_state is None:
                raise ValueError("Initial state must be provided for 'initial' threshold mode.")
            self._check_pattern_shape(initial_state)
            self.thresholds = initial_state.copy()
        else:
            raise ValueError("Invalid threshold_mode specified.")

    def energy(self, state):
        """
        Compute the energy of the current state:
        E = -1/2 * x^T W x + sum_k(theta_k x_k)
        Since W is symmetric and diag(W)=0, this simplifies nicely.
        """
        return -0.5 * state.dot(self.W).dot(state) + np.sum(self.thresholds * state)

    def asynchronous_update(self, state, max_steps=1000, print_interval=1):
        """
        Perform asynchronous update until no state changes or max_steps reached.
        Returns the final state and a list of energy values during recall.

        For small networks (K ≤ 100):
            Print ASCII-art of the state every step.
        For large networks (K > 100):
            Print the energy value every print_interval steps and keep history.

        Parameters:
        -----------
        state : array of shape (K,)
            Initial state. Elements in {-1, +1}.
        max_steps : int
            Maximum number of asynchronous update steps (each step updates all neurons once).
        print_interval : int
            How often to print the output (energy or ASCII-art).
        """
        self._check_pattern_shape(state)
        energy_history = []
        final_states = []

        for step in range(max_steps):
            changed = False

            # Update neurons in random order to avoid bias
            neuron_order = np.random.permutation(self.K)
            for k in neuron_order:
                z_k = np.dot(self.W[k, :], state)
                z_k = z_k - self.thresholds[k]
                new_state = 1 if z_k > 0 else -1 if z_k < 0 else state[k]
                if new_state != state[k]:
                    state[k] = new_state
                    changed = True

            E = self.energy(state)
            energy_history.append(E)

            # For small networks, print ASCII state
            if self.K <= 100:
                if (step % print_interval) == 0:
                    ascii_state = ''.join(['O' if x == 1 else '#' for x in state])
                    print(f"t={step}: {ascii_state}")
            else:
                # For large networks, print energy
                if (step % print_interval) == 0:
                    print(f"t={step}: E={E}")

            # Store the state for potential analysis
            final_states.append(state.copy())

            # Stop if no changes
            if not changed:
                print(f"Converged at step {step}")
                break

        return state, energy_history

# ---------------- Example Usage ---------------- #

# Example for Small Networks (K ≤ 100)

# Define small patterns for demonstration:
pattern1 = np.array([1, 1, 1, -1, -1, -1, 1, 1])    # K=8
pattern2 = np.array([-1, -1, 1, 1, -1, 1, -1, 1])   # Another pattern

# Create a Hopfield Network instance with 8 neurons and zero threshold mode
hop_net = HopfieldNetwork(num_neurons=8, threshold_mode='zero')

# Train the network with the defined patterns
hop_net.train_autoassociator([pattern1, pattern2])

# Create a noisy version of pattern1 by flipping one bit
noisy_pattern = pattern1.copy()
noisy_pattern[2] = -1  # Flip one bit to introduce noise

# Set thresholds based on the initial noisy pattern if needed
# For 'zero' mode, thresholds are already set to zero
hop_net.set_thresholds()

# Perform the asynchronous update using the noisy pattern
final_state, E_hist = hop_net.asynchronous_update(noisy_pattern, max_steps=50, print_interval=1)

# Print the final state of the network after recall
print("Final state:", final_state)
# Optionally, map back to pattern labels or visualize further

# Example for Large Networks (K > 100)

# Parameters
K = 200  # Number of neurons
P = 20   # Number of patterns to store (should be <= 0.14*K ~ 28)

# Generate random patterns: P patterns of K dimensions, values in {-1, +1}
np.random.seed(42)  # For reproducibility
patterns = np.random.choice([-1, 1], size=(P, K))

# Create a Hopfield Network instance with K neurons and zero threshold mode
hop_net = HopfieldNetwork(num_neurons=K, threshold_mode='zero')

# Train the network with the generated patterns
hop_net.train_autoassociator(patterns)

# Select a pattern to recall and create a noisy version
original_pattern = patterns[0]
noisy_pattern = original_pattern.copy()

# Introduce noise by flipping 10% of the bits
num_flips = int(0.1 * K)
flip_indices = np.random.choice(K, size=num_flips, replace=False)
noisy_pattern[flip_indices] *= -1  # Flip selected bits

# Set thresholds (zero in this case)
hop_net.set_thresholds()

# Perform the asynchronous update using the noisy pattern
final_state, E_hist = hop_net.asynchronous_update(noisy_pattern, max_steps=100, print_interval=1)


# Compare final_state with original_pattern
matches = np.sum(final_state == original_pattern)
print(f"Recall Accuracy: {matches}/{K} bits correct")


t=0: OOO###OO
t=1: OOO###OO
Converged at step 1
Final state: [ 1  1  1 -1 -1 -1  1  1]
t=0: E=-20074.0
t=1: E=-20210.0
t=2: E=-20210.0
Converged at step 2
Recall Accuracy: 200/200 bits correct
